# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [ ]:
import torch as th

In [ ]:
x = th.tensor([1,2,3,4,5])
x

In [ ]:
y = x + x

In [ ]:
print(y)

In [ ]:

import syft as sy

In [ ]:
#modifies pytorch
hook = sy.TorchHook(th)

In [ ]:
#pytorch continues to work as before
th.tensor([1,2,3,4,5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [ ]:
#worker simulates interface we have to another machine
bob = sy.VirtualWorker(hook, id="bob")

In [ ]:
#collection of objects simple tensors working with
bob._objects

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.send(bob)

In [ ]:
#Bob's object collection is = {76957029230: tensor([1, 2, 3, 4, 5])}
# this is a pointer to the object, serialized into Json object
bob._objects

In [ ]:
#pointer is pointing to Bob = <VirtualWorker id:bob #objects:1>
x.location

In [ ]:
#pointer location = 76957029230
x.id_at_location

In [ ]:
#this is the id of the object 83032657046
x.id

In [ ]:
#who owns pointer = <VirtualWorker id:me #objects:0>
x.owner

In [ ]:
hook.local_worker

In [ ]:
#local worker, contact bob and tell him to do this
#(Wrapper)>[PointerTensor | me:83032657046 -> bob:76957029230]
x

In [ ]:
#x gets the tensor -should be = tensor([1, 2, 3, 4, 5])
x = x.get()
x

In [ ]:
#result {}
bob._objects

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [ ]:
#import pytoch and create tensor  - pointers to remote tensors
import torch as th
x = th.tensor([1,2,3,4,5])
x

In [ ]:
#import PySyft
#modifies pytorch
import syft as sy
hook = sy.TorchHook(th)

In [ ]:
#create virtual workers bob and alice
#worker simulates interface we have to another machine
bob = sy.VirtualWorker(hook, id="bob")
#worker simulates interface we have to another machine
alice= sy.VirtualWorker(hook, id="alice")

In [ ]:
# y = y.send(bob,alice)

In [ ]:
# y.owner

In [ ]:
# y.location

In [ ]:
# bob._objects

In [ ]:
# alice._objects

In [ ]:
#y = y.get()
# y

In [ ]:
# after adding alice as virtual worker
# x = th.tensor([1,2,3,4,5])
#x_ptr = x.send(bob, alice)   #this is a multipointer
# x_ptr

In [ ]:
x_ptr = x.send(bob, alice)

In [ ]:
# multipointer returns bob and alice
#(Wrapper)>[MultiPointerTensor]
#	-> (Wrapper)>[PointerTensor | me:55988065873 -> bob:9222176640]
#	-> (Wrapper)>[PointerTensor | me:20656805602 -> alice:7139222520]
x_ptr

In [ ]:
#dictionary of virtual workers
#{'bob': (Wrapper)>[PointerTensor | me:55988065873 -> bob:9222176640],
# 'alice': (Wrapper)>[PointerTensor | me:20656805602 -> alice:7139222520]}
# x_ptr.child.child

In [ ]:
#multipointer to bob and alice each pointing to tensor
x_ptr.get()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob,alice)

In [ ]:
#sum results of bob and alice tensors
x.get(sum_results=True)

# Lesson: Introducing Remote Arithmetic

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [ ]:
x

In [ ]:
y

In [ ]:
#sum tensors
z = x + y

In [ ]:
z

In [ ]:
#result of the summed tensors
z = z.get()
z

In [ ]:
#another way of summing result
z = th.add(x,y)
z

In [ ]:
z = z.get()
z

In [ ]:
#backwards propagration creates gradients on x, y
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [ ]:
z = (x + y).sum()

In [ ]:
z.backward()

In [ ]:
x = x.get()

In [ ]:
x

In [ ]:
x.grad

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [ ]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

In [ ]:
#worker simulates interface we have to another machine
bob = sy.VirtualWorker(hook, id="bob")

In [ ]:
#create data for model
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [ ]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [ ]:
#forward propagation - prediction
pred = input.mm(weights)

In [ ]:
#prediction is also a pointer
pred

In [ ]:
loss = ((pred - target)**2).sum()

In [ ]:
#backward propagation
loss.backward()

weights.data.sub_(weights.grad * 0.1)
weights.grad *= 0

In [ ]:
#tensor(2.)
print(loss.get().data)

In [ ]:
#loop to reduce loss as below
'''
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)
tensor(0.0058)
'''
for i in range(10):
    pred = input.mm(weights)
    loss = ((pred - target)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *=0
    print(loss.get().data)
    

# Lesson: Garbage Collection and Common Errors


In [1]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)
#worker simulates interface we have to another machine
bob = sy.VirtualWorker(hook, id="bob")

W0722 04:09:09.263000  8648 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0722 04:09:09.290000  8648 deprecation_wrapper.py:119] From C:\Users\Claudia\Anaconda3\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob = bob.clear_objects()

In [3]:
bob._objects

{}

In [4]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [5]:
bob._objects

{38270506972: tensor([1, 2, 3, 4, 5])}

In [6]:
del x

In [7]:
bob._objects

{}

In [8]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
#result = True - when data is sent this is the default attribute
x.child.garbage_collect_data

In [9]:
bob._objects

{51847049424: tensor([1, 2, 3, 4, 5])}

In [10]:
x = "asdf"

In [11]:
bob._objects

{}

In [13]:
x = th.tensor([1,2,3,4,5]).send(bob)

#gotcha - garbage collection turned off - pointer cached even if deleted


In [15]:
x="asdf"

In [17]:
#bob.objects

del x

In [19]:
#del x still lives on
bob._objects
#####


{}

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
x = "asdf"

In [ ]:
bob._objects

In [ ]:
del x

In [ ]:
bob._objects

In [20]:
#### start here again
bob = bob.clear_objects()
bob._objects

{}

In [21]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [22]:
bob._objects

{91856614208: tensor([1, 2, 3, 4, 5])}

In [23]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [24]:
#Results in PureTorchTensorFoundError - means one is a regulatory tensor
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:19428905153 -> bob:28275297068]
Tensor B: tensor([1, 1, 1, 1, 1])

In [26]:
#Another example of a common error
#worker simulates interface we have to another machine
alice= sy.VirtualWorker(hook, id="alice")
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [27]:
#Result: TensorsNotCollocatedException
z = x + y

TensorsNotCollocatedException: You tried to call __add__ involving two tensors which are not on the same machine! One tensor is on <VirtualWorker id:bob #objects:2> while the other is on <VirtualWorker id:alice #objects:1>. Use a combination of .move(), .get(), and/or .send() to co-locate them to the same machine.

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [ ]:
from torch import nn, optim

In [ ]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [ ]:
# A Toy Model
model = nn.Linear(2,1)

In [ ]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [ ]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

In [ ]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [ ]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [ ]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [ ]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [ ]:
train()

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
x = x.send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
y = x + x

In [ ]:
y

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
jon = sy.VirtualWorker(hook, id="jon")

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = x.get()
x

In [ ]:
bob._objects

In [ ]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
del x

In [ ]:
bob._objects

In [ ]:
alice._objects

# Lesson: Pointer Chain Operations

In [ ]:
bob.clear_objects()
alice.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.remote_get()

In [ ]:
bob._objects

In [ ]:
alice._objects

In [ ]:
x.move(bob)

In [ ]:
x

In [ ]:
bob._objects

In [ ]:
alice._objects